<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/deepcut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install transformers torch datasets

In [5]:
!pip install transformers
!pip install qdrant-client>=1.1.1
!pip install pythainlp

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AIEngineer/data.csv")
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,subject,description,ticket_type,clean_subject,clean_description,tokenized_subject,tokenized_description,meaning_subject,meaning_description
0,Interlink : MA วันที่ 15 กรกฎาคม 2565 เวลา 00....,"From: ""NQM Interlink Telecom"" <nqm@interlinkte...",Request,ma น น cid inextbaac สาขา หน่วยบ...,เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไ...,"['ma', 'น', 'น', 'cid', 'inextbaac', 'สาขา', '...","['เรื่อง', 'ขอ', 'กำหนด', 'การปรับปรุง', 'การ'...","['inextbaac', 'cid', 'สาขา', 'หน่วยบุรีรัมย์',...","['ท่าน', 'สิ้นสุดดำเนินการ', 'ปรับปรุง', 'ขอจั..."
1,[Closed]Promessy Corporation : แจ้งตรวจสอบเครื...,From: Werachat Nawaroongruang <werachat.nawaro...,Request,promessy corporation ตรวจสอบเครื่อง ip r...,inet team เครื่อง private ip ได้มีการ rest...,"['promessy', 'corporation', 'ตรวจสอบ', 'เครื่อ...","['inet', 'team', 'เครื่อง', 'private', 'ip', '...","['ip', 'restart', 'ตรวจสอบเครื่อง', 'โดยไม่ทรา...","['team', 'ได้มีการ', 'รบกวนเช็คให้หน่อยครับ', ..."
2,BAAC : รบกวนเข้าไปเก็บตัวกล่องสัญญาณที่สาขาสาร...,รับสายลูกค้าคุณสุริยา เบอร์ 0818863505 แจ้งมีต...,Request,รบกวนเข้าไปเก็บตัวกล่องสัญญาณที่สาขาสารภี เนื่...,รับสายลูกค้าคุณสุริยา เบอร์ มีตัวกล่องอุปกรณ์...,"['รบกวน', 'เข้าไป', 'เก็บตัว', 'กล่อง', 'สัญญา...","['รับสาย', 'ลูกค้า', 'คุณ', 'สุริยา', 'เบอร์',...","['เข้าไป', 'สารภี', 'เนื่องจากมีการย้ายอาคารไป...","['เข้าไป', 'เดิม', 'โทร', 'เพชร', 'ตัว', 'สุภา..."
3,Transform You : แจ้งขอข้อมูล Lun ดังนี้ Offsit...,"From: ""Teerayut Saowamok"" <teerayut.sa@transfo...",Request,ขอข้อมูล lun ดังนี้ offsite อยู่ที่ idc ไหนและ...,noc รบกวนเปิดเคสประสานงานทีม op cloud ทางทีม t...,"['ขอ', 'ข้อมูล', 'lun', 'ดังนี้', 'offsite', '...","['noc', 'รบกวน', 'เปิด', 'เคส', 'ประสานงาน', '...","['ดังนี้', 'full', 'อยู่ที่', 'ไหนและ', 'offsi...","['full', 'aci', 'เคส', 'sas', 'op', 'offsite',..."
4,GSB[SD] : แบบคำขอเปลี่ยนแปลงทรัพยากรบน GSB Clo...,From: Servicedesk <servicedesk@inetms.co.th>\n...,Request,แบบคำขอเปลี่ยนแปลงทรัพยากรบน gsb cloud based s...,noc ครับ subject internal gsb sr รับ ร...,"['แบบ', 'คำขอ', 'เปลี่ยนแปลง', 'ทรัพยากร', 'บน...","['noc', 'ครับ', 'subject', 'internal', 'gsb', ...","['แบบคำขอเปลี่ยนแปลงทรัพยากรบน', 'กรมการปกครอง...","['อายุการใช้งาน', 'ทรัพยากร', 'gsb', 'gateway'..."


In [21]:
df['sentence'] = df['clean_subject'] + ' ' + df['clean_description']
df['sentence'].loc[0]

'ma     น      น   cid  inextbaac   สาขา หน่วยบุรีรัมย์ เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไอเน็กซ์ บรอดแบนด์ จำกัด ขอจัดส่งจดหมายปรับปรุงการบริการวงจร จำนวน  วงจร วัตถุประ์การดำเนินการ   ปรับปรุงบริการโครงข่าย เนื่องจากจะดำเนินการ replace อุปกรณ์ภายใน node พื้นที่ดำเนินการ   node บุรีรัมย์ site interlink วัน เริ่มดำเนินการ       น  วัน สิ้นสุดดำเนินการ       น  ระยะกระทบ   กระทบบริการไม่เกิน   ชั่วโมง รายละเอียดทั้งหมดตาม เอกสารแนบมา จึงมาโปรดทราบความจำเป็นดังกล่าว และบริษัทฯ ขออภัยในความไม่สะดวกในครั้งนี้ หากท่านมีข้อสงสัยหรือต้องการข้อมูลเพิ่มเติม โปรดติดต่อตามหมายเลขด้านล่าง หรือ reply mail กลับมาที่'

In [22]:
df.head(1)

,subject,description,ticket_type,clean_subject,clean_description,tokenized_subject,tokenized_description,meaning_subject,meaning_description,sentence
0,Interlink : MA วันที่ 15 กรกฎาคม 2565 เวลา 00....,"From: ""NQM Interlink Telecom"" <nqm@interlinkte...",Request,ma น น cid inextbaac สาขา หน่วยบ...,เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไ...,"['ma', 'น', 'น', 'cid', 'inextbaac', 'สาขา', '...","['เรื่อง', 'ขอ', 'กำหนด', 'การปรับปรุง', 'การ'...","['inextbaac', 'cid', 'สาขา', 'หน่วยบุรีรัมย์',...","['ท่าน', 'สิ้นสุดดำเนินการ', 'ปรับปรุง', 'ขอจั...",ma น น cid inextbaac สาขา หน่วยบ...


In [23]:
doc = df[['sentence', 'ticket_type']]
sen_list = doc.to_dict(orient='records')
sen_list[0]

{'sentence': 'ma     น      น   cid  inextbaac   สาขา หน่วยบุรีรัมย์ เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไอเน็กซ์ บรอดแบนด์ จำกัด ขอจัดส่งจดหมายปรับปรุงการบริการวงจร จำนวน  วงจร วัตถุประ์การดำเนินการ   ปรับปรุงบริการโครงข่าย เนื่องจากจะดำเนินการ replace อุปกรณ์ภายใน node พื้นที่ดำเนินการ   node บุรีรัมย์ site interlink วัน เริ่มดำเนินการ       น  วัน สิ้นสุดดำเนินการ       น  ระยะกระทบ   กระทบบริการไม่เกิน   ชั่วโมง รายละเอียดทั้งหมดตาม เอกสารแนบมา จึงมาโปรดทราบความจำเป็นดังกล่าว และบริษัทฯ ขออภัยในความไม่สะดวกในครั้งนี้ หากท่านมีข้อสงสัยหรือต้องการข้อมูลเพิ่มเติม โปรดติดต่อตามหมายเลขด้านล่าง หรือ reply mail กลับมาที่',
 'ticket_type': 'Request'}

In [26]:
!pip install qdrant-client

In [29]:
!pip install deepcut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.2
    Uninstalling protobuf-5.27.2:
      Successfully uninstalled protobuf-5.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
grpcio-tools 1.64.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.3 which is incompatible.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [27]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
from torch import Tensor
from pythainlp.tokenize import word_tokenize
from qdrant_client import models

In [ ]:
def average_pool(last_hidden_states: Tensor, attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Using PyThaiNLP to tokenize the input texts
input_texts = [' '.join(word_tokenize(doc['sentence'], engine="deepcut")) for doc in sen_list]

# Load transformers model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large-instruct')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large-instruct')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

# Get model outputs
outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# Normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

# Connect to Qdrant
qdrant = QdrantClient(":memory:")

# Create a collection in Qdrant
qdrant.recreate_collection(
    collection_name="test",
    vectors_config=models.VectorParams(
        size=embeddings.size(-1),  # Use the size of the embeddings
        distance=models.Distance.COSINE
    )
)

# Prepare points to insert
points = [
    PointStruct(
        id=i,
        vector=embeddings[i].tolist(),
        payload={"document": sen_list[i]}
    )
    for i in range(len(sen_list))
]

# Insert points into the collection
qdrant.upsert(
    collection_name="test",
    points=points
)


Streaming output truncated to the last 5000 lines.
4/9 [============>.................] - ETA: 0s

In [ ]:
# Example query
query = 'สาขา หน่วยบุรีรัมย์ เรื่อง ขอกำหนดการปรับปรุงการให้บริการ บริษัท ไอเน็กซ์ บรอดแบนด์ จำกัด ขอจัดส่งจดหมายปรับปรุงการบริการวงจร จำนวน  วงจร วัตถุประการดำเนินการ'
query_text = ' '.join(word_tokenize(query, engine="deepcut"))
query_dict = tokenizer(query_text, max_length=512, padding=True, truncation=True, return_tensors='pt')

query_outputs = model(**query_dict)
query_embedding = average_pool(query_outputs.last_hidden_state, query_dict['attention_mask'])

query_embedding = F.normalize(query_embedding, p=2, dim=1)

# Search in Qdrant
search_result = qdrant.search(
    collection_name="my_collection",
    query_vector=query_embedding[0].tolist(),
    limit=3  # Number of nearest neighbors to return
)

# Output search results
for i, hit in enumerate(search_result):
    print(f"Result {i+1}:")
    print("Score:", hit.score)
    print("Payload:", hit.payload)
    print(f"--------------------")
